In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse as sp
from scipy import fft
import scipy.sparse.linalg as linalg
import matplotlib.animation as animation
%matplotlib tk

In [ ]:
def laplacian2D(N):
    diag=np.ones([N*N])
    mat=sp.spdiags([diag,-2*diag,diag],[-1,0,1],N,N)
    I=sp.eye(N)
    return sp.kron(I,mat,format='csr')+sp.kron(mat,I, format="csr")#.toarray()

print(laplacian2D(4).toarray())

def plotheatmap(u_k, k, delta_t):
    # Clear the current plot figure
    plt.clf()

    plt.title(f"Temperature at t = {k*delta_t:.3f} unit time")
    plt.xlabel("x")
    plt.ylabel("y")

    # This is to plot u_k (u at time-step k)
    plt.pcolormesh(u_k, cmap="turbo", vmin=300, vmax=1000)
    plt.colorbar()

    return plt

def animate(k, u, delta_t):
    plotheatmap(u[k], k, delta_t)

def dst2(x):
    return fft.dst(fft.dst(x, axis=1), axis=0)

def idst2(x):
    return fft.idst(fft.idst(x, axis=1), axis=0)

[[-4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -4.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  1. -4.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0. -4.  1.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -4.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1. -4. 

In [ ]:
plate_length = 128
max_iter_time = 100

alpha = 2
delta_x = 1

delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)

# Initialize solution: the grid of u(k, i, j)
u = np.empty((max_iter_time, plate_length, plate_length))

# Initial condition everywhere inside the grid
u_initial = 300

# Boundary conditions
u_top = 300.0
u_left = 300.0
u_bottom = 300.0
u_right = 300.0

lin = np.linspace(10,20,5)
x,y = np.meshgrid(lin,lin)
x,y = np.int16(x.flatten()), np.int16(y.flatten())


u.fill(u_initial)
# Set the initial condition
u[:,x, y] = 1000

# Set the boundary conditions
u[:, (plate_length-1):, :] = u_top
u[:, :, :1] = u_left
u[:, :1, 1:] = u_bottom
u[:, :, (plate_length-1):] = u_right

def calculate(u):
    for k in range(0, max_iter_time-1, 1):
        for i in range(1, plate_length-1, delta_x):
            for j in range(1, plate_length-1, delta_x):
                u[:,x, y] = 1000
                u[k + 1, i, j] = gamma * (u[k][i+1][j] + u[k][i-1][j] + u[k][i][j+1] + u[k][i][j-1] - 4*u[k][i][j]) + u[k][i][j]

    return u

u = calculate(u)

anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False, fargs=(u, delta_t))

In [ ]:
plate_length = 128
max_iter_time = 1000

alpha = 1
delta_x = 1

delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)

# Initialize solution: the grid of u(k, i, j)
u = np.random.randint(200,400,(max_iter_time, plate_length, plate_length))

# Initial condition everywhere inside the grid
#u_initial = 300
#u.fill(u_initial)


# Heater element positions
lin = np.linspace(plate_length*0.2,plate_length*0.8,10)
x,y = np.meshgrid(lin,lin)
x,y = np.int16(x.flatten()), np.int16(y.flatten())
idx = (x*plate_length) + y
heater_idx = np.int16(idx)

# Boundary condition positions
idx_matrix = np.arange(0,plate_length**2).reshape(plate_length, plate_length)
idx_top = idx_matrix[(plate_length-1):, :][0]
idx_left = idx_matrix[:, :1][:,0]
idx_bottom = idx_matrix[:1, 1:][0]
idx_right = idx_matrix[:, (plate_length-1):][:,0]
boundary_idx = np.hstack([idx_top, idx_bottom, idx_left, idx_right])

# Get Laplacian matrix
A = laplacian2D(plate_length)#.toarray()
U = u.copy().reshape(max_iter_time,-1)

D = sp.diags(np.ones(plate_length*plate_length)*4, format="csr")
R = A-D
D_inv = sp.diags(np.ones(plate_length*plate_length)/4, format="csr")#linalg.inv(D)

U[:,heater_idx] = 1000
U[:,boundary_idx] = 300


def calculate(U):
    for k in range(0, max_iter_time-1, 1):
        U[k+1] = U[k] + gamma*A@U[k]
        U[:,heater_idx] = 1000
        U[:,boundary_idx] = 300
        if np.allclose(U[k+1], U[k], rtol=0.001):
            print("break after iteration", k)
            break
    #U_k = U[0].copy()
    
    return U.reshape(max_iter_time, plate_length, plate_length)


u = calculate(U).copy()
#print(u[-1])

#plotheatmap(u[1], max_iter_time, delta_t)

anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False, fargs=(u, delta_t))
#anim.save("heat_equation.gif")

In [ ]:
plate_length = 128
max_iter_time = 1000

alpha = 1
delta_x = 1

delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)



# Initialize solution: the grid of u(k, i, j)
u = np.empty((max_iter_time, plate_length, plate_length))

# Initial condition everywhere inside the grid
u_initial = 0#300

# Boundary conditions
u_top = 0#300.0
u_left = 0#300.0
u_bottom = 0#300.0
u_right = 0#300.0

lin = np.linspace(plate_length*0.1,plate_length*0.9,10)
x,y = np.meshgrid(lin,lin)
x,y = np.int16(x.flatten()), np.int16(y.flatten())


u.fill(u_initial)
# Set the initial condition
u[:,x, y] = 700

# Set the boundary conditions
u[:, (plate_length-1):, :] = u_top
u[:, :, :1] = u_left
u[:, :1, 1:] = u_bottom
u[:, :, (plate_length-1):] = u_right

u0 = u.copy()

a = 1    # Thermal diffusivity constant
L = plate_length  # Length of domain
N = plate_length # Number of discretization points
dx = L/N
#x = np.arange(-L/2,L/2,dx) # Define x domain

# Define discrete wavenumbers
kappa = 2*np.pi*np.fft.fftfreq(N, d=dx)
kapx,kapy = np.meshgrid(kappa,kappa)
kap = kapx**2 + kapy**2

delta_t = 0.1

u[0] = u0[0]

for i in range(max_iter_time-1):
    u0hat = np.fft.fft2(u[i]).copy()
    d_uhat = -a * kap * u0hat

    u0hat = u0hat + delta_t*d_uhat

    u[i+1] = np.abs(np.fft.ifft2(u0hat))

    # Set the heater elements
    u[i+1 ,x, y] = 700

    # Set the boundary conditions
    u[i+1, (plate_length-1):, :] = u_top
    u[i+1, :, :1] = u_left
    u[i+1, :1, 1:] = u_bottom
    u[i+1, :, (plate_length-1):] = u_right

u += 300

anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False, fargs=(u, delta_t))

In [8]:
plate_length = 128
max_iter_time = 1000

alpha = 1
delta_x = 1

delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)

# Initialize solution: the grid of u(k, i, j)
u = np.random.randint(200,400,(plate_length, plate_length))


# Boundary conditions
u_top = 300.0
u_left = 300.0
u_bottom = 300.0
u_right = 300.0

lin = np.arange(16, plate_length, 16)#np.linspace(plate_length*,plate_length*0.8,8)
x,y = np.meshgrid(lin,lin)
x,y = x.flatten(), y.flatten()

# Set the initial condition
u[x, y] = 1000

# Set the boundary conditions
u[(plate_length-1):, :] = u_top
u[:, :1] = u_left
u[:1, 1:] = u_bottom
u[:, (plate_length-1):] = u_right



plotheatmap(u, 0, delta_t)
plt.show()

def restrict(u):
    m,n = u.shape[0]//2, u.shape[1]//2
    u_new = np.zeros((m, n))
    print(x,y)
    x_new, y_new = np.int16(x/2), np.int16(y/2)
    print(x_new, y_new)
    
    for i in range(m):
        for j in range(n):
            u_new[i,j] = u[i*2, j*2]
            
    # Set the initial condition
    u_new[x_new, y_new] = 1000

    # Set the boundary conditions
    u_new[(m-1):, :] = u_top
    u_new[:, :1] = u_left
    u_new[:1, 1:] = u_bottom
    u_new[:, (n-1):] = u_right
    
    return u_new, x_new, y_new

def prolongate(u):
    m,n = u.shape[0]*2, u.shape[1]*2
    u_new = np.zeros((m, n))
    x_new, y_new = x*2, y*2
    
    # Set the initial condition
    u_new[x_new, y_new] = 1000

    # Set the boundary conditions
    u_new[(m-1):, :] = u_top
    u_new[:, :1] = u_left
    u_new[:1, 1:] = u_bottom
    u_new[:, (n-1):] = u_right
    
    
    for i in range(2,m-2):
        for j in range(2,n-2):
            u_new[i,j] = (u[int(np.floor((i+1)/2)), int(np.floor((j+1)/2))]
                          + u[int(np.ceil((i+1)/2)), int(np.floor((j+1)/2))]
                          + u[int(np.floor((i+1)/2)), int(np.ceil((j+1)/2))]
                          + u[int(np.ceil((i+1)/2)), int(np.ceil((j+1)/2))])/4
    return u_new, x_new, y_new

    
def jacobi_step(u,x,y):
    m,n = u.shape[0], u.shape[1]
    for i in range(1, m-1, delta_x):
        for j in range(1, n-1, delta_x):
            u[x, y] = 1000
            u[i, j] = gamma * (u[i+1][j] + u[i-1][j] + u[i][j+1] + u[i][j-1] - 4*u[i][j]) + u[i][j]
    return u

def calculate(u,x,y):
    jacobi_steps = 3
    for k in range(jacobi_steps):
        u = jacobi_step(u,x,y)
    u,x,y = restrict(u)
    for k in range(jacobi_steps):
        u = jacobi_step(u,x,y)
    u,x,y = restrict(u)    
    for k in range(jacobi_steps):
        u = jacobi_step(u,x,y)
    u,x,y = prolongate(u)
    for k in range(jacobi_steps):
        u = jacobi_step(u,x,y)
    u,x,y = prolongate(u)
    for k in range(jacobi_steps):
        u = jacobi_step(u,x,y)
    return u

#u = calculate(u,x,y)
#plotheatmap(u, 0, delta_t)
#anim = animation.FuncAnimation(plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False, fargs=(u, delta_t))

In [14]:
jacobi_steps = 3
for k in range(jacobi_steps):
    u = jacobi_step(u,x,y)
u,x,y = restrict(u)
plotheatmap(u,0,delta_t)


[ 32  64  96 128 160 192 224  32  64  96 128 160 192 224  32  64  96 128
 160 192 224  32  64  96 128 160 192 224  32  64  96 128 160 192 224  32
  64  96 128 160 192 224  32  64  96 128 160 192 224] [ 32  32  32  32  32  32  32  64  64  64  64  64  64  64  96  96  96  96
  96  96  96 128 128 128 128 128 128 128 160 160 160 160 160 160 160 192
 192 192 192 192 192 192 224 224 224 224 224 224 224]
[ 16  32  48  64  80  96 112  16  32  48  64  80  96 112  16  32  48  64
  80  96 112  16  32  48  64  80  96 112  16  32  48  64  80  96 112  16
  32  48  64  80  96 112  16  32  48  64  80  96 112] [ 16  16  16  16  16  16  16  32  32  32  32  32  32  32  48  48  48  48
  48  48  48  64  64  64  64  64  64  64  80  80  80  80  80  80  80  96
  96  96  96  96  96  96 112 112 112 112 112 112 112]


<module 'matplotlib.pyplot' from 'c:\\Users\\max-r\\anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py'>

In [13]:
u,x,y = prolongate(u)
plotheatmap(u,0, delta_t)

<module 'matplotlib.pyplot' from 'c:\\Users\\max-r\\anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py'>

In [15]:
696342e3/1e8


6.96342

In [17]:
0.5/0.068

7.352941176470588